# nvImageCodec with cuPy

In [ ]:
import os
import cv2
import cupy as cp
from matplotlib import pyplot as plt

Setting resource folder

In [ ]:
resources_dir = os.getenv("PYNVIMGCODEC_EXAMPLES_RESOURCES_DIR", "../assets/images/")

Import nvImageCodec module and create both Decoder and Encoder

In [ ]:
from nvidia import nvimgcodec
decoder = nvimgcodec.Decoder()
encoder = nvimgcodec.Encoder()

Load jpeg2000 image with nvImageCodec

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")
print(nv_img.__cuda_array_interface__)

Pass nvImageCodec Image to cupy asarray as it accepts \_\_cuda_array_interface\_\_ 

In [ ]:
%%time
cp_img = cp.asarray(nv_img)
print(cp_img.__cuda_array_interface__)

Convert to numpy and show

In [ ]:
np_img4k = cp.asnumpy(cp_img)
plt.imshow(np_img4k)

Lets do some opration on image in GPU using cupyx.scipy

In [ ]:
import cupyx.scipy.ndimage

In [ ]:
cp_img_rotated = cupyx.scipy.ndimage.rotate(cp_img, 90)
cp_img_gaussian = cupyx.scipy.ndimage.gaussian_filter(cp_img, sigma = 5)

print(cp_img_rotated.__cuda_array_interface__)

In [ ]:
np_img = cp.asnumpy(cp_img_rotated)
plt.imshow(np_img)

In [ ]:
np_img = cp.asnumpy(cp_img_gaussian)
plt.imshow(np_img)

Convert cupy Image to nvImageCodec Image using \_\_cuda_array_interface\_\_

In [ ]:
%%time
nv_rotated_img = nvimgcodec.as_image(cp_img_rotated)

Check content

In [ ]:
plt.imshow(nv_rotated_img.cpu())

Save as Jpeg2000

In [ ]:
encoder.write("rotated.j2k", nv_rotated_img)

Load with OpenCv to verify

In [ ]:
image = cv2.imread("rotated.j2k")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

Save cupy image to jpg with nvImageCodec

In [ ]:
nv_img_gaussian = nvimgcodec.as_image(cp_img_gaussian)
encoder.write("gaussian.jpg", nv_img_gaussian)

Read back with OpenCV to verify

In [ ]:
image = cv2.imread("gaussian.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

## DLPack

Passing cuPy ndarray to nvImageCodec using DLPack. cuPy ndarray has both __cuda_array_interface__ and __dlpack__ interface. By default, __cuda_array_interface__ is preferred and will be used when we pass cuPy ndarray to nvImageCodec using as_image function. If we would like to use DLPack, firstly PyCapsule need to be taken from cuPy ndarray and then it needs to be passed to nvImageCodec.

In [ ]:
dlpack_img = cp_img_gaussian.toDlpack()
nv_img_gaussian = nvimgcodec.as_image(dlpack_img)
encoder.write("gaussian_dlpack.jpg", nv_img_gaussian)

In [ ]:
image = cv2.imread("gaussian_dlpack.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)


Passing nvImageCodec Image to cuPy using DLPack

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")
cp_img = cp.from_dlpack(nv_img)

In [ ]:
np_img = cp.asnumpy(cp_img)
plt.imshow(np_img)

Alternatively, there is possibility to firstly take PyCapsule and then pass it to cuPy 

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")
py_cap = nv_img.__dlpack__()
cp_img = cp.from_dlpack(py_cap)

In [ ]:
np_img = cp.asnumpy(cp_img)
plt.imshow(np_img4k)

## Specifying sample_format and color_spec

The `as_image` and `as_images` functions now support optional keyword-only parameters `sample_format` and `color_spec` to override the automatic format inference.


### Default format inference

By default, the format is inferred based on the number of channels:


In [ ]:
# Create test images with different channel counts
img_1ch = cp.random.randint(0, 255, (100, 100, 1), dtype=cp.uint8)
img_3ch = cp.random.randint(0, 255, (100, 100, 3), dtype=cp.uint8)
img_4ch = cp.random.randint(0, 255, (100, 100, 4), dtype=cp.uint8)

# Convert to nvImageCodec images - formats are inferred automatically
nv_img_1ch = nvimgcodec.as_image(img_1ch)
nv_img_3ch = nvimgcodec.as_image(img_3ch)
nv_img_4ch = nvimgcodec.as_image(img_4ch)

print(f"1 channel: {nv_img_1ch.sample_format} - {nv_img_1ch.color_spec}")
print(f"3 channels: {nv_img_3ch.sample_format} - {nv_img_3ch.color_spec}")
print(f"4 channels: {nv_img_4ch.sample_format} - {nv_img_4ch.color_spec}")


### Overriding format inference

You can explicitly specify the `sample_format` and `color_spec` (they must be keyword arguments):


In [ ]:
# Override to BGR format for a 3-channel image
nv_img_bgr = nvimgcodec.as_image(img_3ch, sample_format=nvimgcodec.SampleFormat.I_BGR)
print(f"3-channel with BGR override: {nv_img_bgr.sample_format}")

# Override color_spec to SYCC
nv_img_sycc = nvimgcodec.as_image(img_3ch, color_spec=nvimgcodec.ColorSpec.SYCC)
print(f"3-channel with SYCC color spec: {nv_img_sycc.color_spec}")

# Override both
nv_img_custom = nvimgcodec.as_image(img_3ch, 
                                    sample_format=nvimgcodec.SampleFormat.I_YUV,
                                    color_spec=nvimgcodec.ColorSpec.SYCC)
print(f"Custom format: {nv_img_custom.sample_format} - {nv_img_custom.color_spec}")


### Format validation

The library validates that the specified `sample_format` is compatible with the number of channels. For example, RGBA format requires at least 4 channels:


In [ ]:
# This will raise an error: trying to use RGBA format with only 3 channels
try:
    nv_img_invalid = nvimgcodec.as_image(img_3ch, sample_format=nvimgcodec.SampleFormat.I_RGBA)
except Exception as e:
    print(f"Error (as expected): {e}")

# However, having MORE channels than required is allowed
# (only the first 3 channels will be used for RGB)
nv_img_rgb_from_4ch = nvimgcodec.as_image(img_4ch, sample_format=nvimgcodec.SampleFormat.I_RGB)
print(f"4-channel image with RGB format: {nv_img_rgb_from_4ch.sample_format}")


### Using with as_images

The same parameters work with `as_images` for batch processing:


In [ ]:
# Convert multiple images at once with custom format
images = nvimgcodec.as_images([img_1ch, img_3ch, img_4ch], 
                              sample_format=nvimgcodec.SampleFormat.I_Y,
                              color_spec=nvimgcodec.ColorSpec.GRAY)

for i, img in enumerate(images):
    print(f"Image {i}: shape={img.shape}, sample_format={img.sample_format}, color_spec={img.color_spec}")
